# AI Workshop - Lab 3-1a: Optimization
## Supply Chain Logistics Optimization using Google OR-Tools

This notebook will guide you through using Google OR-Tools to solve a supply chain optimization problem.
We'll use Linear Programming (LP) to minimize transportation costs in a supply chain.

## Problem Overview
Imagine a company that needs to transport goods from warehouses to stores. Each warehouse has a limited supply, and each store requires a certain amount of goods. Additionally, transporting goods incurs costs, which depend on the warehouse-store route. Our goal is to determine the optimal transportation plan to minimize total costs.

We'll use Google OR-Tools to solve this problem. OR-Tools is an open-source library for combinatorial optimization, which includes linear programming, constraint programming, and vehicle routing.

In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp

## Problem Data

We'll use the following data for our problem:

- Warehouses: the supply locations.
- Stores: the demand locations.
- Costs: the transportation costs between warehouses and stores.
- Supplies: the available supply at each warehouse.
- Demands: the required demand at each store.

We'll represent this data using Python dictionaries. Fill out these values with your own data.

In [ ]:
# Data

# Warehouses - a list of supply locations
warehouses = ['Warehouse A', 'Warehouse B']

# Stores
stores = ['Store 1', 'Store 2', 'Store 3']

# Costs
costs = {
    ('Warehouse A', 'Store 1'): 1,
    ('Warehouse A', 'Store 2'): 2,
    ('Warehouse A', 'Store 3'): 3,
    ('Warehouse B', 'Store 1'): 3,
    ('Warehouse B', 'Store 2'): 2,
    ('Warehouse B', 'Store 3'): 1,
}

# Supplies
supplies = {
    'Warehouse A': 100,
    'Warehouse B': 150,
}

# Demands
demands = {
    'Store 1': 50,
    'Store 2': 100,
    'Store 3': 75,
}

# Verify that the costs, supplies and demands are complete
assert len(warehouses) * len(stores) == len(costs), 'Costs are incomplete'
assert len(warehouses) == len(supplies), 'Supplies are incomplete'
assert len(stores) == len(demands), 'Demands are incomplete'

## Model

Next, we'll define the optimization model using Google OR-Tools. We'll create a linear programming model to minimize the total transportation costs.

Google OR-Tools provides a Python API for defining linear programming models. We can create variables, constraints, and the objective function using this API. We'll be using the GLOP linear solver, which is a high-performance solver for linear programming problems.

Let's define the model.

In [ ]:
# Model

# Create the linear solver with the GLOP backend
solver = pywraplp.Solver.CreateSolver('GLOP')

First we need to define the "decision variables" for the model. These variables represent the amount of goods transported from each warehouse to each store. We'll create a variable for each warehouse-store pair. Some of these might be zero if there is no route between a warehouse and a store.

In [ ]:
# Decision Variables

# Create a variable for each warehouse-store pair
variables = {}
for warehouse in warehouses:
    for store in stores:
        variables[warehouse, store] = solver.NumVar(0, solver.infinity(), f'{warehouse} to {store}')

# Verify the variables
variables

Notice that we defined the _bounds_ of each variable to be between 0 and infinity. This means that the amount of goods transported can be any non-negative value - anywhere from zero to all the goods available at the warehouse.

# Constraints

Next, we need to define the constraints for the model. These constraints ensure that the total amount of goods transported from each warehouse does not exceed the available supply, and that the total amount of goods received at each store meets the required demand. We'll create two sets of constraints: one for supplies and one for demands.

In [ ]:
# Supply Constraints: total goods transported from each warehouse <= available supply
for warehouse in warehouses:
    solver.Add(sum(variables[warehouse, store] for store in stores) <= supplies[warehouse])

# Demand Constraints: total goods received at each store >= required demand
for store in stores:
    solver.Add(sum(variables[warehouse, store] for warehouse in warehouses) >= demands[store])

# Verify the constraints
solver.NumConstraints()

# Objective Function

Finally, we need to define the objective function for the model. This tells our solver what it is we're trying to optimize. In this case, we want to minimize the total transportation costs, which is the sum of costs for each warehouse-store pair. Notice that we've already specified that each store must receive at least the required demand, and each warehouse cannot exceed its available supply.

In [ ]:
# Objective Function: minimize total transportation costs
for warehouse in warehouses:
    for store in stores:
        solver.Minimize(variables[warehouse, store] * costs[warehouse, store])

Now that we've defined the model, we can run the solver to find the optimal solution! We'll call the `Solve` method on the solver object to solve the linear programming problem.

In [ ]:
# Solve the model
status = solver.Solve()

In [ ]:
# Check if the problem has an optimal solution
if status == pywraplp.Solver.OPTIMAL:
    print('Optimal Solution Found')
else:
    print('The problem does not have an optimal solution')

Fantastic - the solver found an optimal solution! Now let's print out the optimal transportation plan and the total transportation costs.

In [ ]:
# Print the optimal transportation plan
total_cost = 0
for warehouse in warehouses:
    for store in stores:
        amount = variables[warehouse, store].solution_value()
        if amount > 0:
            cost = amount * costs[warehouse, store]
            total_cost += cost
            print(f'Transport {amount} units from {warehouse} to {store} at a cost of {cost}')

print(f'\nTotal Transportation Cost: {total_cost}')

# Summary

In this notebook, we used Google OR-Tools to solve a supply chain optimization problem. We defined a linear programming model to minimize transportation costs in a supply chain, considering the available supply at each warehouse and the required demand at each store. We then used the GLOP linear solver to find the optimal transportation plan that minimizes total costs.

You can modify the data and constraints to test different scenarios and see how the optimal solution changes. Linear programming is a powerful tool for optimization problems, and Google OR-Tools makes it easy to define and solve these models in Python.

